In [1]:
from __future__ import annotations
import argparse
import numpy as np
import pandas as pd
import sys, os
sys.path.append(os.path.relpath("../src/"))
from dataloader import S1, S2, S3, S4
from dataset import EpicDataset
import torch
from torch.utils.data import DataLoader, Dataset
from sequential import SequenceEncoder

import pytorch_lightning as L
from pytorch_lightning import seed_everything, Trainer
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

In [2]:
torch.random.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Torch version: {torch.__version__}")
print(f"GPU?: {device}")

Torch version: 2.0.0
GPU?: cpu


In [3]:
def getDatasetForScenario(n: int, modality: str) -> Dataset:
    if n == 1:
        dataset = EpicDataset(S1(), modality)
    elif n == 2:
        dataset = EpicDataset(S2(), modality)
    elif n == 3:
        dataset = EpicDataset(S3(), modality)
    else: #n == 4:
        dataset = EpicDataset(S4(), modality)
    return dataset


class LitModel(L.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def training_step(self, batch):
        x, y = batch
        x = x.view(x.size(0), -1)
        y_hat = self.model(x)
        loss = torch.nn.functional.mse_loss(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

In [4]:
seed_everything(1)
checkpoint_dir = "checkpoints"
checkpoint = ModelCheckpoint(
    checkpoint_dir,
    monitor="Losses/val_loss",
    mode="min",
    save_top_k=5,
    save_weights_only=False,
    verbose=True,
)

Global seed set to 1


In [5]:
train_checkpoint = ModelCheckpoint(
        checkpoint_dir,
        monitor="Losses/train_loss",
        mode="min",
        save_top_k=5,
        save_weights_only=False,
        verbose=True,
)

In [6]:
lr_monitor = LearningRateMonitor(logging_interval="step")
callbacks = [
    checkpoint,
    train_checkpoint,
    lr_monitor,
]

In [17]:
exp_dir = "exp"
epochs = 1
nodes = 1

trainer = Trainer(
        default_root_dir=exp_dir,
        max_epochs=epochs,
        num_nodes=nodes,
        # gpus=args.gpus, # all?
        # accelerator="gpu",
#        strategy=DDPPlugin(find_unused_parameters=False),
        callbacks=callbacks,
        reload_dataloaders_every_n_epochs=1,
        gradient_clip_val=10.0,
    )

In [8]:
numIn = 50
hidden = 32
out = 2
model = SequenceEncoder(numIn, hidden, out)

In [9]:
scenario = 1
data_module = DataLoader(getDatasetForScenario(scenario, 'ecg'), batch_size=1, shuffle=False, num_workers=4)

In [ ]:
trainer.fit(LitModel(model), data_module)#, ckpt_path=checkpoint_dir)


  | Name  | Type            | Params
------------------------------------------
0 | model | SequenceEncoder | 14.5 K
------------------------------------------
14.5 K    Trainable params
0         Non-trainable params
14.5 K    Total params
0.058     Total estimated model params size (MB)
/Users/felixdollack/miniconda3/envs/pytorch/lib/python3.9/site-packages/lightning_fabric/loggers/csv_logs.py:188: UserWarning: Experiment logs directory exp/lightning_logs/version_0 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  rank_zero_warn(
/Users/felixdollack/miniconda3/envs/pytorch/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.

Training: 0it [00:00, ?it/s]